<a href="https://colab.research.google.com/github/rahulhalderAERO/Hydrodynamic_LSTM_ROM/blob/main/Hydrodynamic_PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The LSTM-PINN algorithms introduces the loss function computed from the discretized governing equation.The Governing Equation is the rigid body dynamic Equation as $M\ddot{x}  = F$


In [1]:
from keras.models import Sequential 
from keras.layers import Dense
import numpy as np
from matplotlib import pyplot as plt
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from keras import backend as K
import tensorflow as tf
import random
import time
start_time = time.time()
Ext_Col = 6
test_step = 25

Import Training and Test Data

In [ ]:
Output_Train = pd.read_csv("box_smooth_Dis.csv",skiprows = None , header = None )
Input_Train = pd.read_csv("Test_Input_Force_Only.csv",skiprows = None , header = None )

Output_Test = pd.read_csv("box_smooth_Dis.csv",skiprows = None , header = None )
Input_Test = pd.read_csv("Test_Input_Force_Only.csv",skiprows = None , header = None )

Normalize the input data

In [ ]:
Output_array = Output_Train.values
input_scaler = MinMaxScaler(feature_range=(0, 1))
input_Traindata = input_scaler.fit_transform(Input_Train)
input_Testdata = input_scaler.fit_transform(Input_Test)

initialize output with zero




In [ ]:
output_discrete = np.zeros(shape=(Output_array.shape[0],Output_array.shape[1]+Ext_Col*2))
x_train = []
y_train = []
x_test = []

Prepare the input train dataset with the windows of time

In [ ]:
for i in range(test_step,input_Traindata.shape[0]):
    x_train.append(input_Traindata[i-test_step:i,0].reshape(-1,1))
x_train_mid = np.array(x_train)
x_train = []

for j in range(2,4):
    for i in range(test_step,input_Traindata.shape[0]):
        x_train.append(input_Traindata[i-test_step:i,j].reshape(-1,1))
    x_train_mid1 = np.array(x_train)
    x_train = []
    x_train_final = np.concatenate((x_train_mid,x_train_mid1),axis = 2) 
    x_train_mid = x_train_final
    
x_train = x_train_final

Prepare the input test dataset with the windows of time


In [ ]:
for i in range(test_step,input_Testdata.shape[0]):
    x_test.append(input_Testdata[i-test_step:i,0].reshape(-1,1))
x_test_mid = np.array(x_test)
x_test = []

for j in range(2,4):
    for i in range(test_step,input_Testdata.shape[0]):
        x_test.append(input_Testdata[i-test_step:i,j].reshape(-1,1))
    x_test_mid1 = np.array(x_test)
    x_test = []
    x_test_final = np.concatenate((x_test_mid,x_test_mid1),axis = 2) 
    x_test_mid = x_test_final
    
x_test = x_test_final

Prepare the output dataset with the windows of time

In [ ]:
for j in range(Ext_Col):
    output_discrete[:,3*(j)] = Output_array[:,j]
    output_discrete[1:,3*(j)+1] = Output_array[:-1,j]
    output_discrete[2:,3*(j)+2] = Output_array[:-2:,j]
output_final =  np.concatenate((output_discrete,input_Traindata[:,1:4]),axis=1)


output_scaler = MinMaxScaler(feature_range=(0, 1))
output_Traindata = output_scaler.fit_transform(output_final)


y_train = output_Traindata[test_step:len(output_final)]

Take percentage of data

In [ ]:
list=[]
for i in range(1):
    r=random.randint(1,y_train.shape[0]-1)
    if r not in list: list.append(r)
    list_array = np.array(list)


for k in range(len(list_array)):
    val_row = list_array[k]
    y_train[val_row,:-3] = 0

LSTM Model


In [ ]:
model = Sequential()


model.add(LSTM(units = 50,return_sequences = True, input_shape = (x_train.shape[1], x_train.shape[2])))    
model.add(LSTM(units =50,return_sequences = True))
model.add(LSTM(units = 50))
model.add(Dense(units = 21))

Train and prediction Phase

In [ ]:
epochs = 50
model.compile(optimizer = 'adam' , loss = PINN_LOSS)
history = model.fit(x_train,y_train, epochs = epochs , batch_size = 25)
A = history.history['loss']
print("--- %s seconds ---" % (time.time() - start_time))
predicted_y = (model.predict(x_train))
y_pred_trainset_inv = output_scaler.inverse_transform(predicted_y)

plot

In [ ]:
plt.plot(np.arange(0,len(y_train)),Output_array[test_step:,1],label='Actual')
plt.plot(np.arange(0,len(y_train)),y_pred_trainset_inv[:,3] , label ='Predicted')